In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils
import utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_filter_map_plot

from IPython.display import display, Markdown, Latex, HTML
import json

In [2]:
## parameters cell
itp_id = 300

In [3]:
# Parameters
itp_id = 350


In [4]:
airtable_organizations = (
    tbl.airtable.california_transit_organizations()
    >> select(_.itp_id, _.name, _.caltrans_district,
              _.website, _.ntp_id, _.drmt_organization_name)
    >> filter(_.itp_id == itp_id)
    >> collect()
)

In [5]:
# airtable_organizations

In [6]:
analysis_date = dt.date(2022, 3, 30)

In [7]:
trips = pd.read_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{itp_id}_03_30.parquet')

In [8]:
stop_delay = gpd.read_parquet(f'{utils.GCS_FILE_PATH}stop_delay_views/{itp_id}_03_30.parquet')
stop_delay['arrival_time'] = stop_delay.arrival_time.map(lambda x: dt.datetime.fromisoformat(x))
stop_delay['actual_time'] = stop_delay.actual_time.map(lambda x: dt.datetime.fromisoformat(x))

In [9]:
%%capture
routelines = utils.get_routelines(itp_id, analysis_date)

In [10]:
%%capture
rt_day = rt_filter_map_plot.RtFilterMapper(trips, stop_delay, routelines)

In [11]:
%%capture_parameters
agency_name = rt_day.calitp_agency_name
district = airtable_organizations.caltrans_district.iloc[0]
agency_name, district

# Bus Speed and Delay for Union City Transit, District 04 - Oakland

In [12]:
## avoid papermill errors when map fails to render
m = None

## Morning Peak, March 30

In [13]:
%%capture
rt_day.set_filter(start_time='06:00', end_time='09:00')

In [14]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds', no_title=True)

In [15]:
m

In [16]:
rt_day.describe_delayed_routes()

AM Peak most delayed routes: 
 * 5, Union Landing, Four Corners, Dyer & Union City Blvd, and Union City BART, Eastbound: 2.0 minutes late on average
 * 2, Decoto District, Whipple Rd, Kaiser, Continues as Route 8 to Union Landing, Perry Rd, and Union City BART, Eastbound: 1.0 minutes late on average
 * 2, Decoto District, Whipple Rd, Kaiser, Continues as Route 8 to Union Landing, Perry Rd, and Union City BART, Westbound: 1.0 minutes late on average
 * 3, Tropics, Union Landing, Continues as Route 4 to Calaveras Landing, Whipple Rd, Seven Hills, and Union City BART, Westbound: 1.0 minutes late on average
 * 1, Union Landing, Four Corners, Dyer & Union City Blvd, and Union City BART, Westbound: 1.0 minutes late on average

## Midday, March 30

In [17]:
%%capture
rt_day.set_filter(start_time='10:00', end_time='14:00')

In [18]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds', no_title=True)

In [19]:
m

In [20]:
rt_day.describe_delayed_routes()

Midday most delayed routes: 
 * 1, Union Landing, Four Corners, Dyer & Union City Blvd, and Union City BART, Westbound: 3.0 minutes late on average
 * 1, Union Landing, Four Corners, Dyer & Union City Blvd, and Union City BART, Eastbound: 3.0 minutes late on average
 * 5, Union Landing, Four Corners, Dyer & Union City Blvd, and Union City BART, Westbound: 2.0 minutes late on average
 * 5, Union Landing, Four Corners, Dyer & Union City Blvd, and Union City BART, Eastbound: 1.0 minutes late on average
 * 2, Decoto District, Whipple Rd, Kaiser, Continues as Route 8 to Union Landing, Perry Rd, and Union City BART, Eastbound: 1.0 minutes late on average

## Afternoon Peak, March 30

In [21]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00')

In [22]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds', no_title=True)

In [23]:
m

In [24]:
rt_day.describe_delayed_routes()

PM Peak most delayed routes: 
 * 1, Union Landing, Four Corners, Dyer & Union City Blvd, and Union City BART, Eastbound: 7.0 minutes late on average
 * 3, Tropics, Union Landing, Continues as Route 4 to Calaveras Landing, Whipple Rd, Seven Hills, and Union City BART, Eastbound: 6.0 minutes late on average
 * 3, Tropics, Union Landing, Continues as Route 4 to Calaveras Landing, Whipple Rd, Seven Hills, and Union City BART, Westbound: 4.0 minutes late on average
 * 4, Seven Hills, Whipple Rd, Calaveras Landing, Union Landing, Continues as Route 3 to Tropics and Union City BART, Eastbound: 4.0 minutes late on average
 * 2, Decoto District, Whipple Rd, Kaiser, Continues as Route 8 to Union Landing, Perry Rd, and Union City BART, Eastbound: 3.0 minutes late on average

In [25]:
try:
    metrics_df = (rt_day.rt_trips
         >> group_by(_.route_short_name, _.direction_id)
         >> summarize(speed_variance = _.mean_speed_mph.var(),
                     mean_speed_mph = _.mean_speed_mph.mean(),
                     min_hour = _.median_time.min().hour,
                     max_hour = _.median_time.max().hour)
         >> filter(_.min_hour <= 7, _.max_hour >= 15,
                   _.mean_speed_mph < _.mean_speed_mph.quantile(.6),
                   _.speed_variance)
         >> arrange(-_.speed_variance)
         >> head(20)
        )
    metrics_df
except Exception as e:
    print(e)
## check that route has all-day span...

In [26]:
def map_from_metrics(rt_day, metrics_df):
    try:
        route_name = metrics_df.iloc[1,0]
        direction_id = metrics_df.iloc[1,1]
    except Exception as e:
        print(e)
        return
    # route_name = '281'
    # direction_id = '1'
    rt_day.set_filter(start_time='07:00', end_time='09:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        am_map = rt_day.segment_speed_map(how='low_speeds')
        display(am_map)
    except Exception as e:
        print(e)
        pass
    
    rt_day.set_filter(start_time='11:00', end_time='14:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        mid_map = rt_day.segment_speed_map(how='low_speeds')
        display(mid_map)
    except Exception as e:
        print(e)
        pass

    rt_day.set_filter(start_time='15:00', end_time='19:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        pm_map = rt_day.segment_speed_map(how='low_speeds')
        display(pm_map)
    except Exception as e:
        print(e)
        pass

    return

## Single-Route Detail Example

In [27]:
try:
    map_from_metrics(rt_day, metrics_df)
except Exception as e:
    print(e)
    pass

speed above 80 for shape p_1274523, dropping
